# Importing Libraries

In [1]:
%%configure -f
{"conf":{
"spark.pyspark.python": "python27",
"spark.pyspark.virtualenv.enabled": "true",
"spark.pyspark.virtualenv.type":"native",
"spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
}}

In [2]:
import findspark                                              #Import library to Search for Spark Installation

findspark.init()                                              #Search Spark Installation

import pyspark                                                #Only run after findspark.init()

from pyspark.sql import SparkSession                          #Import of Spark Session
from pyspark import SparkContext as spark                     #Import the Regular Spark Contex
from pyspark.sql import SQLContext                            #Import the SQL Spark Contex
from pyspark.sql.functions import *                           #Import the SQL Spark Functions
from pyspark.sql.types import *                               #Import the SQL Spark Variables Types
spark = SparkSession.builder.getOrCreate()                    #Creating Spark Session
import os

from datetime import date                                     # Import date methoed
month_ref = date.today().month                                # Creating Current Month Variable
year_ref = date.today().year                                  # Creating Current Year Variable

sc = spark.sparkContext                                       #Initialize Spark Session

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1573728220057_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-----------------------------

# Creating Variables

## Buckets

In [ ]:
SPG_MANUAL_INPUT_BUCKET = os.environ['MANUAL_INPUT_BUCKET']

In [ ]:
SPG_INTEGRATION_INPUT_BUCKET = os.environ['INTEGRATION_INPUT_BUCKET']

In [ ]:
SPG_QUERY_BUCKET = os.environ['QUERY_BUCKET']

In [ ]:
GERDAU_BUCKET = os.environ['GERDAU_BUCKET']

In [ ]:
SPG_OUTPUT_BUCKET = os.environ['OUTPUT_BUCKET']

In [ ]:
SPG_INPUT_BUCKET = os.environ['INPUT_BUCKET_FROM_OUTPUT']

## Input Paths

In [ ]:
# SPG SAP Hana
SPG_INTEGRATION_INPUT_BUCKET_HANA = "SPG_FACTS/SPG_HANA/SPG_HANA.parquet"

In [ ]:
# SPG Regions
SPG_INTEGRATION_INPUT_BUCKET_REGION = "SPG_DIMENSIONS/SPG_REGIOES/SPG_RG_REGIOES/SPG_RG_REGIOES.parquet"

In [ ]:
# SPG Products
SPG_INTEGRATION_INPUT_BUCKET_PRODUCTS = "SPG_DIMENSIONS/SPG_PRODUTOS/SPG_PD_PRODUTOS.parquet"

In [ ]:
# SPG Gross Profit
SPG_INPUT_BUCKET_GROSS_PROFIT = "SPG_LB/LB_OrdemItem"

## Output Paths

In [ ]:
SPG_OUTPUT_BUCKET_4_PRICES = "SPG_4_PRICES/SPG_4P_4_PRICES/tb_SPG_4P_4_PRICES.parquet"

--------------

# Creating Defined

In [3]:
def normalizing_orthography(col_name):
    removed_array = [('Á', 'A'), ('Ã', 'A'),('À', 'A'),('Â', 'A'),('Ä', 'A'),
                    ('É', 'E'),('È', 'E'),('Ê', 'E'),('Ë', 'E'),
                    ('Í', 'I'),('Ì', 'I'),('Î', 'I'),('Ï', 'I'),
                    ('Ó', 'O'),('Õ', 'O'), ('Ò', 'O'),('Ô', 'O'),('Ö', 'O'),
                    ('Ú', 'U'),('Ù', 'U'),('Û', 'U'),('Ü', 'U'),
                    ('Ç', 'C'),('`', ''),("'", ''),('-', ''),('/', ''),(' ', '')]
    r = col_name
    for a, b in removed_array:
        r = regexp_replace(r, a, b)
    return r

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

----------------------------------------------

# Importing Tables

In [4]:
# SPG SAP Hana
df_hana = spark.read.parquet("s3a://"+SPG_INTEGRATION_INPUT_BUCKET+"/"+SPG_INTEGRATION_INPUT_BUCKET_HANA)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# SPG Regions
df_regions = spark.read.parquet("s3a://"+SPG_INTEGRATION_INPUT_BUCKET+"/"+SPG_INTEGRATION_INPUT_BUCKET_REGION)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# SPG Products
df_products = spark.read.parquet("s3a://"+SPG_INTEGRATION_INPUT_BUCKET+"/"+SPG_INTEGRATION_INPUT_BUCKET_PRODUCTS)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# SPG Gross Profit
df_gross_profit = spark.read.parquet("s3a://"+SPG_INPUT_BUCKET+"/"+SPG_INPUT_BUCKET_GROSS_PROFIT)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

----------------------------------------

# Preparing the Tables

## Preparing Gross Profit

In [8]:
df_gross_profit = df_gross_profit.withColumn("MONTH_YEAR_LB", concat(when(col("month") >= "10", col("month"))
                                                                        .otherwise(concat(lit("0"), col("month")))
                                                                    ,lit("-")
                                                                    ,col("year")))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
df_gross_profit = df_gross_profit.withColumn("key_Lb_drop", concat(col("MONTH_YEAR_LB")
                                                                   ,lit("_")
                                                                   ,col("sales_number").cast("int")
                                                                   ,lit("_")
                                                                   ,col("sales_item").cast("int")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
df_gross_profit=df_gross_profit.select(df_gross_profit.key_Lb_drop.alias("key_Lb_drop")
                                       ,df_gross_profit.COGS_TOTAL.alias("OP_COST").cast('float')
                                       ,df_gross_profit.FREIGHT_Full.alias("FREIGHT").cast('float')
                                       ,df_gross_profit.GROSS_PROFIT_BW.alias("GROSS_PROFIT_LB").cast('float'))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
df_gross_profit.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[key_Lb_drop: string, OP_COST: float, FREIGHT: float, GROSS_PROFIT_LB: float]

## Preparing Region

In [12]:
df_regions=df_regions.withColumn("UPPER_NAME", normalizing_orthography(upper(df_regions["IBGE_SAP_CITY_NAME"])))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
df_regions=df_regions.select(df_regions.UPPER_NAME
                             ,df_regions.BRANCH
                             ,df_regions.IBGE_UF_ACRONYMS)\
                    .dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
df_regions.write.partitionBy("IBGE_UF_ACRONYMS")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
df_regions = df_regions.repartition("IBGE_UF_ACRONYMS")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
df_regions.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[UPPER_NAME: string, BRANCH: string, IBGE_UF_ACRONYMS: string]

--------------

## Preparing Products

In [17]:
df_products=df_products.select(trim(df_products.DESC_GPD).alias("DESC_GPD")
                               ,df_products.COD_GPD
                               ,trim(df_products.DESC_GPM).alias("GPM_DESC_prod_drop")
                               ,df_products.COD_MATERIAL.alias("COD_MATERIAL_drop")
                               ,trim(df_products.SALES_ORG_COD).alias("SALES_ORG_COD_drop"))\
                        .dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
df_products.write.partitionBy("GPM_DESC_prod_drop")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
df_products = df_products.repartition("GPM_DESC_prod_drop")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

u"cannot resolve '`DESC_GPM`' given input columns: [COD_GPD, SALES_ORG_COD_drop, COD_MATERIAL_drop, DESC_GPD, GPM_DESC_prod_drop];;\n'RepartitionByExpression ['DESC_GPM], 200\n+- Deduplicate [COD_GPD#597, COD_MATERIAL_drop#896, SALES_ORG_COD_drop#897, DESC_GPD#894, GPM_DESC_prod_drop#895]\n   +- Project [trim(DESC_GPD#599, None) AS DESC_GPD#894, COD_GPD#597, trim(DESC_GPM#650, None) AS GPM_DESC_prod_drop#895, COD_MATERIAL#592 AS COD_MATERIAL_drop#896, trim(SALES_ORG_COD#598, None) AS SALES_ORG_COD_drop#897]\n      +- Relation[COD_MATERIAL#592,COD_ACTIVITY_SECTOR#593,MARKED_DELETION#594,DESC_MATERIAL#595,DESC_ACTIVITY_SECTOR#596,COD_GPD#597,SALES_ORG_COD#598,DESC_GPD#599,BAG#600,BAR#601,CS#602,DR#603,EA#604,FT#605,FT2#606,GLL#607,KAR#608,KG#609,KI#610,KWH#611,LB#612,LBF#613,M#614,M2#615,... 37 more fields] parquet\n"
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py", line 737, in repartition
    return DataFrame(self._jdf.repartit

In [20]:
df_products.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[DESC_GPD: string, COD_GPD: string, GPM_DESC_prod_drop: string, COD_MATERIAL_drop: string, SALES_ORG_COD_drop: string]

---------------

## Preparing Hana

In [21]:
# Filtering Sales Org and Billing Type
df_hana=df_hana.filter(df_hana.SalesOrg.like('%BRIN%') |
                       df_hana.SalesOrg.like('%BRIO%') |
                       df_hana.SalesOrg.like('%BRDI%') |
                       df_hana.SalesOrg.like('%BRDO%') |
                       df_hana.SalesOrg.like('%BRCC%') |
                       df_hana.SalesOrg.like('%BRCO%') |
                       df_hana.SalesOrg.like('%BRCG%') |
                       df_hana.SalesOrg.like('%BRGO%'))\
               .filter(df_hana.BillingType.like('%ZF2B%'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# Correcting "Ouro Branco" Sales Org 
    # BRIO -> BRIN
    # BRCO -> BRCC
    # BRDO -> BRDI
    # BRGO -> BRCG
df_hana = df_hana.withColumn("SalesOrg", when(df_hana.SalesOrg.like("BRCO"), "BRCC")\
                                              .otherwise(when(df_hana.SalesOrg.like("BRGO"), "BRCG")\
                                                        .otherwise(when(df_hana.SalesOrg.like("BRIO"), "BRIN")\
                                                                  .otherwise(when(df_hana.SalesOrg.like("BRDO"), "BRDI")
                                                                            .otherwise(df_hana.SalesOrg)))))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
# Getting GPM_DESC Field
df_hana=df_hana.join(df_products, (trim(df_products.SALES_ORG_COD_drop)==trim(df_hana.SalesOrg))&
                                  (df_products.COD_MATERIAL_drop.cast("int")==df_hana.Material.cast("int")), how="left")\
               .drop("COD_MATERIAL_drop")\
               .drop("SALES_ORG_COD_drop")\
               .dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
# Correcting GPM_DESC Column
df_hana=df_hana.withColumn("GPM", when(df_hana.GPM_DESC_prod_drop.isNull(), df_hana.GPM)\
                                           .otherwise(df_hana.GPM_DESC_prod_drop))\
               .withColumn("GPD", when(df_hana.COD_GPD.isNull(), df_hana.GPD)\
                                           .otherwise(df_hana.COD_GPD))\
               .withColumn("GPDName", when(df_hana.DESC_GPD.isNull(), df_hana.GPDName)\
                                           .otherwise(df_hana.DESC_GPD))\
               .drop("GPM_DESC_prod_drop")\
               .drop("DESC_GPD")\
               .drop("COD_GPD")\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
df_hana=df_hana.withColumn("GPM", when(df_hana.GPM.isNull(), "Não Informado")\
                                           .otherwise(df_hana.GPM))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
#selecting specifics GPD's (business call)
df_hana = df_hana.filter(df_hana.GPDName.like('%BR-PERFIS ESTRUTURAIS%') |
                        df_hana.GPDName.like('%BR-CA50%') |
                        df_hana.GPDName.like('%BR-CA60%') |
                        df_hana.GPDName.like('%BR-RECOZIDO%') |
                        df_hana.GPDName.like('%BR-TEL TUBO%') |
                        df_hana.GPDName.like('%BR-TELA P/ CONCRETO%') |
                        df_hana.GPDName.like('%BR-CORTE E DOBR%') |
                        df_hana.GPDName.like('%BR-CONSTRUCAO CIVIL%') |
                        df_hana.GPDName.like('%BR-CA25%') |
                        df_hana.GPDName.like('%BR-PERFIS COMERCIAIS%') |
                        df_hana.GPDName.like('%BR-BTG%') |
                        df_hana.GPDName.like('%BR-MALHA POP%') |
                        df_hana.GPDName.like('%BR-TELA COLUNA%') |
                        df_hana.GPDName.like('%BR-TRELIÇA%') |
                        df_hana.GPDName.like('%BR-B.TREFILADA%') |
                        df_hana.GPDName.like('%BR-OVALADO%') |
                        df_hana.GPDName.like('%BR-GALVANIZADO%') |
                        df_hana.GPDName.like('%BR-BARRAS COMERCIAIS%') |
                        df_hana.GPDName.like('%BR-CAIXOTARIA%') |
                        df_hana.GPDName.like('%BR-ARTEFATOS%') |
                        df_hana.GPDName.like('%BR-FARPADO%') |
                        df_hana.GPDName.like('%BR-ELETRODO%') |
                        df_hana.GPDName.like('%BR-SOLDA - MIG%') |
                        df_hana.GPDName.like('%BR-CANTON A572%') |
                        df_hana.GPDName.like('%BR-GGS%') |
                        df_hana.GPDName.like('%BR-ARMADO%') |
                        df_hana.GPDName.like('%BR-CORDOALHA%') |
                        df_hana.GPDName.like('%BR-ESTACA PRANCHA%') |
                        df_hana.GPDName.like('%BR-ARAME PREGO%') |
                        df_hana.GPDName.like('%BR-CABEÇA DUPLA%') |
                        df_hana.GPDName.like('%BR-CORDOALHA AGRO%') |
                        df_hana.GPDName.like('%BR-GRAMPO%') |
                        df_hana.GPDName.like('%BR-COBREADOS%') |
                        df_hana.GPDName.like('%BR-CHAPA LQ%') |
                        df_hana.GPDName.like('%BR-UDC%') |
                        df_hana.GPDName.like('%BR-CHAPA ZN%') |
                        df_hana.GPDName.like('%BR-TELHA AZ%') |
                        df_hana.GPDName.like('%BR-TUBO ZN%') |
                        df_hana.GPDName.like('%BR-MARCENARIA%') |
                        df_hana.GPDName.like('%BR-PREGOES%') |
                        df_hana.GPDName.like('%BR-TELHEIRO%') |
                        df_hana.GPDName.like('%BR-COLUNA%') |
                        df_hana.GPDName.like('%BR-ESTRIBO%') |
                        df_hana.GPDName.like('%BR-ACESSORIOS%') |
                        df_hana.GPDName.like('%BR-CHAPA LCG%') |
                        df_hana.GPDName.like('%BR-CHAPA LF%') |
                        df_hana.GPDName.like('%BR-TUBO LF%') |
                        df_hana.GPDName.like('%BR-CHAPA LQ PISO%') |
                        df_hana.GPDName.like('%BR-BOBININHA%') |
                        df_hana.GPDName.like('%BR-ESPECIAIS%') |
                        df_hana.GPDName.like('%BR-BOBINA LQ%') |
                        df_hana.GPDName.like('%BR-FITA LQ%') |
                        df_hana.GPDName.like('%BR-BOBINA AZ%') |
                        df_hana.GPDName.like('%BR-AÇOS ESPECIAIS%') |
                        df_hana.GPDName.like('%BR-PARAFUSOS%') |
                        df_hana.GPDName.like('%BR-CIMENTO%') |
                        df_hana.GPDName.like('%BR-TUBO LQ%') |
                        df_hana.GPDName.like('%BR-TELHA ZN%') |
                        df_hana.GPDName.like('%BR-BTC GLV CP%') |
                        df_hana.GPDName.like('%BR-BOBINA ZN%') |
                        df_hana.GPDName.like('%BR-S-BARRA LAMINADA MÉDIA%') |
                        df_hana.GPDName.like('%BR-S-BARRA LAMINADA PESADA%') |
                        df_hana.GPDName.like('%BR-FITA LF%') |
                        df_hana.GPDName.like('%BR-FITA AZ%') |
                        df_hana.GPDName.like('%BR-S-BARRA LAMINADA LEVE%') |
                        df_hana.GPDName.like('%BR-ALAMBRADO%') |
                        df_hana.GPDName.like('%BR-SAPATA%') |
                        df_hana.GPDName.like('%BR-MOURÃO%') |
                        df_hana.GPDName.like('%BR-ATC CLARO IND%') |
                        df_hana.GPDName.like('%BR-POLIDO%') |
                        df_hana.GPDName.like('%BR-PERFIL BENEFICIADO%') |
                        df_hana.GPDName.like('%BR-BOBINA LF%') |
                        df_hana.GPDName.like('%BR-S-BARRA CTT%') |
                        df_hana.GPDName.like('%BR-CHAPA AZ%') |
                        df_hana.GPDName.like('%BR-FITA ZN%') |
                        df_hana.GPDName.like('%BR-LDA%') |
                        df_hana.GPDName.like('%BR-PIATINA CLARA%') |
                        df_hana.GPDName.like('%BR-S-BARRA DESCASCADA%') |
                        df_hana.GPDName.like('%BR-MESH%') |
                        df_hana.GPDName.like('%BR-S-BARRA FORJADA FINA%') |
                        df_hana.GPDName.like('%BR-S-BARRA RETIFICADA%') |
                        df_hana.GPDName.like('%BR-S-BARRA TREFILADA%') |
                        df_hana.GPDName.like('%BR-PLACA%') |
                        df_hana.GPDName.like('%BR-COIL%') |
                        df_hana.GPDName.like('%BR-S-FERRAMENTA%') |
                        df_hana.GPDName.like('%BR-S-FIO MÁQUINA STT%') |
                        df_hana.GPDName.like('%BR-ATC CLARO MOL COL%') |
                        df_hana.GPDName.like('%BR-ATC GLV CP%') |
                        df_hana.GPDName.like('%BR-B.CHATA LONGARINA%') |
                        df_hana.GPDName.like('%BR-BT FORJARIA BLOCOS%') |
                        df_hana.GPDName.like('%BR-TARUGO%') |
                        df_hana.GPDName.like('%BR-DRAWING%') |
                        df_hana.GPDName.like('%BR-ATC CLARO ENF ALG%') |
                        df_hana.GPDName.like('%BR-PIATINA GLV%') |
                        df_hana.GPDName.like('%BR-PERFIL DORMENTE%') |
                        df_hana.GPDName.like('%BR-ATC%') |
                        df_hana.GPDName.like('%BR-CHQ%') |
                        df_hana.GPDName.like('%BR-ALMA DE ELETRODO%') |
                        df_hana.GPDName.like('%BR-FIO MAQUINA%') |
                        df_hana.GPDName.like('%BR-CHQ BORO%') |
                        df_hana.GPDName.like('%BR-PERFIL GUIA ELEV%'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
#selecting specific GPM
df_hana = df_hana.filter(df_hana.GPM.like('%PERFIS_ESTRUTURAIS%') |
                        df_hana.GPM.like('%VERGALHAO%') |
                        df_hana.GPM.like('%AMPLIADOS%') |
                        df_hana.GPM.like('%CORTE_DOBRA%') |
                        df_hana.GPM.like('%PREGOS%') |
                        df_hana.GPM.like('%B&P%') |
                        df_hana.GPM.like('%ARAMES_AGRO%') |
                        df_hana.GPM.like('%ARAMES_IND%') |
                        df_hana.GPM.like('%PLANOS_LQ%') |
                        df_hana.GPM.like('%PLANOS_REVESTIDOS%') |
                        df_hana.GPM.like('%PLANOS_LF%') |
                        df_hana.GPM.like('%PLANOS_LCG%') |
                        df_hana.GPM.like('%FIO_MAQUINA%') |
                        df_hana.GPM.like('%BT_FORJARIA%') |
                        df_hana.GPM.like('%PLACAS%') |
                        df_hana.GPM.like('%TARUGO%'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
# Selecting Necessary Columns
df_hana=df_hana.select(df_hana.ShipToCustomer.alias("CLIENT_COD")
                       ,df_hana.ShipToCustomerName.alias("CLIENT_DESC")
                       ,df_hana.CorporateGroupName.alias("CLIENT_GROUP_DESC")
                       ,df_hana.BILDOCDATE.alias("BILLING_DATE")
                       ,df_hana.SalesOrg.alias("SALES_ORG_COD")
                       ,df_hana.SalesDocNumber.alias("SALES_DOC_NUMBER")
                       ,df_hana.SalesDocItem.alias("SALES_DOC_ITEM")
                       ,df_hana.SoldToCity.alias("ISSUING_CITY")
                       ,df_hana.SoldToState.alias("ISSUING_STATE")
                       ,df_hana.GPD.alias("GPD_COD")
                       ,df_hana.GPM.alias("GPM_DESC")
                       ,df_hana.GPDName.alias("GPD_DESC")
                       ,df_hana.Material.alias("MATERIAL_COD").cast("int").cast("string")
                       ,df_hana.MaterialName.alias("MATERIAL_DESC")
                       ,df_hana.MA_BillingQuantity_P.alias("QUANTITY_TON")
                       ,df_hana.DP_CM_PRECOPRATICADO_D.alias("PRACTICED_PRICE")
                       ,df_hana.DA_CM_CD_ZD05_D.alias("ZD05")
                       ,df_hana.AA_CM_CD_ZEAF_D.alias("ZEAF")
                       ,df_hana.MA_CM_CD_BX41_D.alias("BX41")
                       ,df_hana.MA_CM_RBV_D.alias("RBV")
                       ,df_hana.MA_CM_RLV_D.alias("RLV")
                       ,df_hana.IM_CM_CD_ICMS_D.alias("ICMS")
                       ,df_hana.IM_CM_CD_PIS_D.alias("PIS")
                       ,df_hana.IM_CM_CD_COFINS_D.alias("COFINS")
                       ,df_hana.IM_OUTROS_D.alias("OTHER_TAXES")
                       ,df_hana.IM_CM_CD_BX95_D.alias("BX95"))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
df_hana.write.partitionBy("GPM_DESC")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
df_hana = df_hana.repartition("GPM_DESC")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
df_hana.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[CLIENTE_COD: string, CLIENTE_DESC: string, CLIENT_GROUP_DESC: string, BILLING_DATE: string, SALES_ORG_COD: string, SALES_DOC_NUMBER: string, SALES_DOC_ITEM: string, ISSUING_CITY: string, ISSUING_STATE: string, GPD_COD: string, GPM_DESC: string, GPD_DESC: string, MATERIAL_COD: string, MATERIAL_DESC: string, QUANTITY_TON: double, PRACTICED_PRICE: double, ZD05: double, ZEAF: double, BX41: double, RBV: double, RLV: double, ICMS: double, PIS: double, COFINS: double, OTHER_TAXES: double, BX95: double]

In [32]:
df_hana=df_hana.filter(df_hana['BILLING_DATE']>=add_months(trunc(current_date(),"month"), -6))\
                       .filter(df_hana['BILLING_DATE']<add_months(trunc(current_date(),"month"), 0))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
df_hana = df_hana.withColumn("MONTH_YEAR", concat(when(month(df_hana.BILLING_DATE) >= "10", month(df_hana.BILLING_DATE))
                                                            .otherwise(concat(lit("0"), month(df_hana.BILLING_DATE)))
                                                ,lit("-")
                                                ,year(df_hana.BILLING_DATE) ))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
df_hana=df_hana.withColumn("CLIENT_GROUP_DESC", when(df_hana.CLIENT_GROUP_DESC.isNull()|
                                                     df_hana.CLIENT_GROUP_DESC.like("GRUPO DE RENTABILIDADE GENÉRICO"),col("CLIENT_DESC"))\
                                                   .otherwise(col("CLIENT_GROUP_DESC")))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
for col_name in ["ZD05"
                 ,"ZEAF"
                 ,"ICMS"
                 ,"PIS"
                 ,"COFINS"
                 ,"OTHER_TAXES"
                 ,"BX95"
                 ,"BX41"
                 ,"RBV"
                 ,"RLV"
                 ,"QUANTITY_TON"
                 ,"PRACTICED_PRICE"]:
    df_hana = df_hana.withColumn(col_name, col(col_name).cast('float'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
df_hana=df_hana.filter(df_hana.QUANTITY_TON > 0)\
                     .withColumn("SALES_ORG_COD", trim(df_hana.SALES_ORG_COD))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
df_hana = df_hana.join(df_regions, (normalizing_orthography(upper(df_hana["ISSUING_CITY"])) == df_regions["UPPER_NAME"])&
                                   (trim(df_hana["ISSUING_STATE"]) == trim(df_regions["IBGE_UF_ACRONYMS"])), how='left')\
                     .drop("UPPER_NAME")\
                     .drop("IBGE_UF_ACRONYMS")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
df_hana.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[CLIENTE_COD: string, CLIENTE_DESC: string, CLIENT_GROUP_DESC: string, BILLING_DATE: string, SALES_ORG_COD: string, SALES_DOC_NUMBER: string, SALES_DOC_ITEM: string, ISSUING_CITY: string, ISSUING_STATE: string, GPD_COD: string, GPM_DESC: string, GPD_DESC: string, MATERIAL_COD: string, MATERIAL_DESC: string, QUANTITY_TON: float, PRACTICED_PRICE: float, ZD05: float, ZEAF: float, BX41: float, RBV: float, RLV: float, ICMS: float, PIS: float, COFINS: float, OTHER_TAXES: float, BX95: float, MONTH_YEAR: string, BRANCH: string]

In [39]:
df_hana = df_hana.withColumn("BRANCH", when(df_hana.BRANCH.isNull(), lit("Filial Não Encontrada"))\
                                                   .otherwise(df_hana.BRANCH))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
df_hana = df_hana.withColumn("PRACTICED_PRICE_PER_KG", when((df_hana.SALES_ORG_COD.like("BRCG")), 
                                                            col("PRACTICED_PRICE")+abs(col("ZD05"))-col("ZEAF")+col("BX41"))\
                                                          .otherwise(col("PRACTICED_PRICE")+abs(col("ZD05"))-col("ZEAF")))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
df_hana=df_hana.drop("SALES_ORG_DESC"
                     ,"PRACTICED_PRICE"
                     ,"PRECO_BASE_ALCADA"
                     ,"SALES_DISCOUNT_TAXES"
                     ,"GDD_DESC"
                     ,"GPP_DESC"
                     ,"ZP30"
                     ,"ZP20"
                     ,"CUSTOMER_GROUP_DESC"
                     ,"ISSUING_CITY")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
df_hana = df_hana.withColumn("key_Lb_drop", concat(col("MONTH_YEAR")
                                                ,lit("_")
                                                ,col("SALES_DOC_NUMBER").cast("int")
                                                ,lit("_")
                                                ,col("SALES_DOC_ITEM").cast("int")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
df_client = df_hana.select(df_hana.CLIENT_COD
                           ,df_hana.CLIENT_DESC
                           ,df_hana.CLIENT_GROUP_DESC
                           ,df_hana.key_Lb_drop)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
df_client = df_client.dropDuplicates(["key_Lb_drop"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
df_hana=df_hana.withColumn("PPKG", col("PRACTICED_PRICE_PER_KG")*col("QUANTITY_TON"))\
               .groupBy(df_hana.SALES_ORG_COD
                        ,df_hana.ISSUING_STATE
                        ,df_hana.SALES_DOC_NUMBER
                        ,df_hana.SALES_DOC_ITEM
                        ,df_hana.GPD_COD
                        ,df_hana.GPD_DESC
                        ,df_hana.MATERIAL_COD
                        ,df_hana.MATERIAL_DESC
                        ,df_hana.GPM_DESC
                        ,df_hana.BRANCH
                        ,df_hana.MONTH_YEAR
                        ,df_hana.key_Lb_drop)\
                    .agg(sum(col("PPKG")).alias("PPKG")
                         ,sum(col("ZD05")).alias("ZD05")
                         ,sum(col("ZEAF")).alias("ZEAF")
                         ,sum(col("BX41")).alias("BX41")
                         ,sum(col("RLV")).alias("RLV")
                         ,sum(col("RBV")).alias("RBV")
                         ,sum(col("ICMS")).alias("ICMS")
                         ,sum(col("PIS")).alias("PIS")
                         ,sum(col("COFINS")).alias("COFINS")
                         ,sum(col("OTHER_TAXES")).alias("OTHER_TAXES")
                         ,sum(col("BX95")).alias("BX95")
                         ,sum(col("QUANTITY_TON")).alias("QUANTITY_TON"))\
                .withColumn("PRACTICED_PRICE_PER_KG", col("PPKG")/col("QUANTITY_TON"))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
df_hana.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[SALES_ORG_COD: string, ISSUING_STATE: string, SALES_DOC_NUMBER: string, SALES_DOC_ITEM: string, GPD_COD: string, GPD_DESC: string, MATERIAL_COD: string, MATERIAL_DESC: string, GPM_DESC: string, BRANCH: string, MONTH_YEAR: string, key_Lb_drop: string, PPKG: double, ZD05: double, ZEAF: double, BX41: double, RLV: double, RBV: double, ICMS: double, PIS: double, COFINS: double, OTHER_TAXES: double, BX95: double, QUANTITY_TON: double, PRACTICED_PRICE_PER_KG: double]

In [47]:
df_hana = df_hana.join(df_gross_profit.select(df_gross_profit.key_Lb_drop
                                              ,df_gross_profit.GROSS_PROFIT_LB
                                              ,df_gross_profit.FREIGHT
                                              ,df_gross_profit.OP_COST)
                             ,on = ["key_Lb_drop"], how='left')\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
df_hana.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[key_Lb_drop: string, SALES_ORG_COD: string, ISSUING_STATE: string, SALES_DOC_NUMBER: string, SALES_DOC_ITEM: string, GPD_COD: string, GPD_DESC: string, MATERIAL_COD: string, MATERIAL_DESC: string, GPM_DESC: string, BRANCH: string, MONTH_YEAR: string, PPKG: double, ZD05: double, ZEAF: double, BX41: double, RLV: double, RBV: double, ICMS: double, PIS: double, COFINS: double, OTHER_TAXES: double, BX95: double, QUANTITY_TON: double, PRACTICED_PRICE_PER_KG: double, GROSS_PROFIT_LB: float, FREIGHT: float, OP_COST: float]

In [49]:
df_hana = df_hana.join(df_client,on = ["key_Lb_drop"], how='left')\
                 .drop("key_Lb_drop")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
df_hana.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[SALES_ORG_COD: string, ISSUING_STATE: string, SALES_DOC_NUMBER: string, SALES_DOC_ITEM: string, GPD_COD: string, GPD_DESC: string, MATERIAL_COD: string, MATERIAL_DESC: string, GPM_DESC: string, BRANCH: string, MONTH_YEAR: string, PPKG: double, ZD05: double, ZEAF: double, BX41: double, RLV: double, RBV: double, ICMS: double, PIS: double, COFINS: double, OTHER_TAXES: double, BX95: double, QUANTITY_TON: double, PRACTICED_PRICE_PER_KG: double, GROSS_PROFIT_LB: float, FREIGHT: float, OP_COST: float, CLIENTE_COD: string, CLIENTE_DESC: string, CLIENT_GROUP_DESC: string]

In [51]:
df_hana = df_hana.fillna({'GROSS_PROFIT_LB':'0'
                         ,'FREIGHT':'0'
                         ,'OP_COST':'0'})\
                 .dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
#Creating TAXES
df_hana=df_hana.withColumn("CALCULATED_TAXES",(col('ICMS')
                                              + col('COFINS')
                                              + col("PIS")
                                              + col("OTHER_TAXES")
                                              + col("BX95")).cast('float'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
df_hana = df_hana.withColumn("GROSS_PROFIT_LB", df_hana["GROSS_PROFIT_LB"].cast("float"))\
                 .withColumn("FREIGHT", df_hana["FREIGHT"].cast("float"))\
                 .withColumn("OP_COST", df_hana["OP_COST"].cast("float"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
df_hana = df_hana.withColumn("GROSS_PROFIT_ORIGINAL", col('GROSS_PROFIT_LB'))\
                 .withColumn("FREIGHT_ORIGINAL", col('FREIGHT'))\
                 .withColumn("OPERATIONAL_COST_ORIGINAL", col('OP_COST'))\
                 .withColumn("PRACTICED_PRICE_PER_KG_ORIGINAL", col('PRACTICED_PRICE_PER_KG'))\
                 .withColumn("RLV_ORIGINAL", col('RLV'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
df_hana = df_hana.drop("PPKG")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
# Dividing by quantity and by ton
for col_name in ["GROSS_PROFIT_LB"
                 ,"PRACTICED_PRICE_PER_KG"
                 ,"CALCULATED_TAXES"
                 ,"ZD05"
                 ,"ZEAF"
                 ,"ICMS"
                 ,"PIS"
                 ,"COFINS"
                 ,"OTHER_TAXES"
                 ,"BX95"
                 ,"BX41"
                 ,"RBV"
                 ,"RLV"
                 ,"FREIGHT"
                 ,"OP_COST"]:
    df_hana = df_hana.withColumn(col_name, (col(col_name)/col("QUANTITY_TON")).cast('float'))\
                     .withColumn(col_name, (col(col_name)/1000).cast('float'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

----------------

# Creating Support Columns

In [57]:
df_hana = df_hana.withColumn("COUNTRY", lit("Brazil"))\
                 .dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
df_hana.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[SALES_ORG_COD: string, ISSUING_STATE: string, SALES_DOC_NUMBER: string, SALES_DOC_ITEM: string, GPD_COD: string, GPD_DESC: string, MATERIAL_COD: string, MATERIAL_DESC: string, GPM_DESC: string, BRANCH: string, MONTH_YEAR: string, ZD05: float, ZEAF: float, BX41: float, RLV: float, RBV: float, ICMS: float, PIS: float, COFINS: float, OTHER_TAXES: float, BX95: float, QUANTITY_TON: double, PRACTICED_PRICE_PER_KG: float, GROSS_PROFIT_LB: float, FREIGHT: float, OP_COST: float, CLIENTE_COD: string, CLIENTE_DESC: string, CLIENT_GROUP_DESC: string, CALCULATED_TAXES: float, GROSS_PROFIT_ORIGINAL: float, FREIGHT_ORIGINAL: float, OPERATIONAL_COST_ORIGINAL: float, PRACTICED_PRICE_PER_KG_ORIGINAL: double, RLV_ORIGINAL: double, COUNTRY: string]

------------------

# Uploading Table

In [60]:
df_hana.write.parquet("s3a://"+SPG_OUTPUT_BUCKET+"/"+SPG_OUTPUT_BUCKET_4_PRICES, mode = "overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…